# Correlation between tasks being performed by the target and its power consumption
In this course we will use the `Simple serial` firmware to show how the power consumption in a microcontroller will depend on the kind or instructions it is performing.

**Goals:**
* Capturing a power trace with ChipWhisperer
* Making connections between various simple instructions and power traces

#### Prerequisites
- [x] *0_series* notebooks

## Setup
In order to make the target perform different instructions, we hardcoded those instructions into its source code. The power measurements for different instructions were already measured for you. The following code block will load the correct data from the files and store them in the correct variable:

In [ ]:
import numpy as np
trace_no_specific_instruction = np.load('./../src/sim_files/1A_1-trace_no_specific_instruction.npy')
trace_additions_no_for_loop = np.load('./../src/sim_files/1A_2-trace_additions_no_for_loop.npy')
trace_additions_with_for_loop = np.load('./../src/sim_files/1A_3-trace_additions_with_for_loop.npy')
trace_powers_with_for_loop =np.load('./../src/sim_files/1A_4-trace_powers_with_for_loop.npy')

Let's first look at a measured power trace when no specific instructions are being executed:

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
trace = trace_no_specific_instruction
plt.figure()
plt.plot(trace, 'r')
plt.show()

As you can tell from the graph above, the microcontroller does not seem to be doing a lot. To change this we dove into the source code of simple_serial and we added some nonesense instructions. 

## Changing the Simple serial base source code
In this part of the code something was inserted:

```C
    #if SS_VER == SS_VER_2_0
    uint8_t get_pt(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t* pt)
    #else
    uint8_t get_pt(uint8_t* pt, uint8_t len)
    #endif

    {
        /**********************************
        * Start user-specific code here. */
        trigger_high();

        //16 hex bytes held in 'pt' were sent
        //from the computer. Store your response
        //back into 'pt', which will send 16 bytes
        //back to computer. Can ignore of course if
        //not needed

        trigger_low();
        /* End user-specific code here. *
        ********************************/
        simpleserial_put('r', 16, pt);
        return 0x00;
    }
```

**The following code was inserted right before the line `simpleserial_put('r', 16, pt);`**

```C

	volatile long int A = 0x2BAA;
	A += 2;
	A += 2;
	A += 2;
	A += 2;
	A += 2;
	
	A += 2;
	A += 2;
	A += 2;
	A += 2;
	A += 2;

	A += 2;
	A += 2;
	A += 2;
	A += 2;
	A += 2;
	
	A += 2;
	A += 2;
	A += 2;
	A += 2;
	A += 2;
```
The code below plots the powertrace measured with the new source code:

In [ ]:
trace = trace_additions_no_for_loop
plt.figure()
plt.plot(trace, 'r')
plt.show()

You should see a different graph than you did before. Furthermore, you should see a pattern in the beginning of the trace that repeats approximately 20 times. You get 3 seconds to figure out what the cause of this repeating pattern could be ...

1,2,3

...of course it can be related to the same instruction `A += 2;` that we made the microcontroller perform 20 times.
That doesn't seem that hard to understand but it is a fundamental part in understanding side-channel analysis attacks. We can learn something about the inner workings of a target by just monitoring its power consumption.

#### let's mix it up a bit
Every programmer knows that most of the times if we want to perform a piece of code 20 times we don't just copy the code 20 times. Next, we let the target perform the same 20 instructions. This time we use a for-loop. 
```C
    volatile long int A = 0x2BAA;
    for (int i = 0;i<20;i++){
        A += 2;
    }  

```
The code block below shows the measured trace again:

In [ ]:
trace = trace_additions_with_for_loop
plt.figure()
plt.plot(trace, 'r')
plt.show()

**Notice that the trace is slighly different while we are still performing the same 20 additions. Also notice that the total amount of 'time' is takes to complete all 20 instructions is longer this time!**

*-TIP: if you need to scroll a lot between the graphs, you can simply clear some output of code blocks. Select the cell which output you want to clear and in the menu above select: `Cell -> Current Outputs -> Clear`*. Or select the block (blue border, not green) and press `o` to toggle the output on and off.

The longer time can be explained by the extra overhead a microcontroller needs to perform to do a for-loop instead of just running 20 instructions after each other.

#### More expensive instructions.
Now that we have seen what the power consumption of the target looks like while performing a simple instruction (can be performed in one clock cycle), let's look at a more expensive instruction for the XMEGA. The instruction inside the for loop was changed to this:
```C
    A = A*A;
```

And yet again the code below plots the power trace:

In [ ]:
trace = trace_powers_with_for_loop
plt.figure()
plt.plot(trace, 'r')
plt.show()

This time the instruction takes so long that the chipwhisperer wasn't even able to capture all 20 instructions in 5000 power measurements.

**HINT: In order to measure current, ChipWhisperer measures voltage drop across a shunt resistor. This means that the power trace is actually inverted (aka large negative swings are areas of higher power consumption).**

## Next step
Hopefully you now clearly understand that the instructions a target (microcontroller) performs correlates to the power consumption of the device. In the next lesson we will use this leakage of information to our advantage and find the hidden key of an asymmetric cryptographic algorithm.

**Next notebook click here: [1_B - Attack on RSA(SIMULATED).ipynb](1_B%20-%20Attack%20on%20RSA(SIMULATED).ipynb)**
